In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
def get_details(link):
    soup = BeautifulSoup(requests.get(link).text, 'lxml')
    try:
        details = soup.find('div', 'css-bgzo2k').text
        district = soup.find_all('a', class_ = 'css-tyi2d1')[-1].text.split()[-1]
        tags = [x.text for x in soup.select('li.css-1r0si1e')]
    except AttributeError:
        return ('delete',  '', '')
    return (process_post(details), district, tags)

def process_post(post):
    #Cleans post text. Joins into single string, removes /n
    post_str = "".join(post)
    return post_str.replace('\n', ' ').replace('\r', ' ')

In [3]:
links = pd.read_csv('Data/rent_links.csv')
links_filled=pd.read_parquet('Data/rent_details.parquet')
links_new=links[~links.id.isin(links_filled.id.values)]
properties_details=[]

In [4]:
properties_details=[]
for index, row in tqdm(links_new.iterrows(), total=links_new.shape[0]):
    property = [row.id]
    property.extend(get_details(row.url))
    properties_details.append(property)

  0%|          | 0/2864 [00:00<?, ?it/s]

In [5]:
properties_data = pd.DataFrame(properties_details, columns=['id', 'details','district', 'tags'])

In [6]:
# update links on 'delete' id's
to_del=properties_data[properties_data.details == 'delete']
links[~links.id.isin(to_del.id.values)].to_csv('Data/rent_links.csv', index=False)
# remove delete from links_new. only then concat
properties_data.drop(to_del.index, inplace=True)

In [7]:
new_data=links_new.merge(properties_data, on='id')

In [8]:
links_upd = pd.concat([links_filled, new_data])
links_upd.to_parquet('Data/rent_details.parquet', index=False)